# Kernel methods 

### Import des fonctions


In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV,StratifiedKFold
from sklearn.metrics import accuracy_score
from scipy.linalg import solve,lstsq
from scipy.stats import uniform
from cvxopt import matrix
from cvxopt import solvers
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from itertools import compress, product
import numexpr as ne

In [2]:
X_0=pd.read_csv("/Users/elliotmuller/Documents/M2/Kernel Methods/Data Challenge/Xtr0_mat100.csv",sep=' ',header=None)
X_1=pd.read_csv("/Users/elliotmuller/Documents/M2/Kernel Methods/Data Challenge/Xtr1_mat100.csv",sep=';',header=None)
X_2=pd.read_csv("/Users/elliotmuller/Documents/M2/Kernel Methods/Data Challenge/Xtr2_mat100.csv",sep=' ',header=None)


Y_0=pd.read_csv("/Users/elliotmuller/Documents/M2/Kernel Methods/Data Challenge/Ytr0.csv",sep=',')
Y_1=pd.read_csv("/Users/elliotmuller/Documents/M2/Kernel Methods/Data Challenge/Ytr1.csv",sep=',')
Y_2=pd.read_csv("/Users/elliotmuller/Documents/M2/Kernel Methods/Data Challenge/Ytr2.csv",sep=',')

Y_0

,Id,Bound
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
1995,1995,0
1996,1996,1
1997,1997,0
1998,1998,0


In [3]:
print(Y_0["Bound"].value_counts())
print(Y_1["Bound"].value_counts())
print(Y_2["Bound"].value_counts())

0    1038
1     962
Name: Bound, dtype: int64
1    1001
0     999
Name: Bound, dtype: int64
0    1003
1     997
Name: Bound, dtype: int64


Les classes sont plutôt équilibrées 

Remplacer les 0 par des -1

In [4]:
Y_0train=np.where(Y_0["Bound"]==0,-1,Y_0["Bound"])
Y_1train=np.where(Y_1["Bound"]==0,-1,Y_1["Bound"])
Y_2train=np.where(Y_2["Bound"]==0,-1,Y_2["Bound"])

X_0train=X_0.to_numpy()
X_1train=X_1.to_numpy()
X_2train=X_2.to_numpy()
print(X_0train.shape)

(2000, 100)


In [5]:
X_1

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.000000,0.000000,0.010870,0.010870,0.000000,0.021739,0.032609,0.021739,0.000000,0.021739,...,0.000000,0.021739,0.021739,0.065217,0.000000,0.000000,0.032609,0.043478,0.021739,0.010870
1,0.010870,0.000000,0.000000,0.000000,0.043478,0.010870,0.010870,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.021739,0.021739,0.010870,0.000000,0.000000,0.032609,0.021739,0.010870
2,0.021739,0.000000,0.000000,0.021739,0.000000,0.010870,0.021739,0.010870,0.000000,0.021739,...,0.010870,0.010870,0.000000,0.010870,0.000000,0.000000,0.021739,0.000000,0.021739,0.010870
3,0.000000,0.000000,0.000000,0.021739,0.000000,0.010870,0.000000,0.010870,0.000000,0.000000,...,0.021739,0.000000,0.010870,0.010870,0.000000,0.000000,0.010870,0.010870,0.021739,0.000000
4,0.000000,0.010870,0.000000,0.010870,0.000000,0.000000,0.010870,0.021739,0.000000,0.021739,...,0.000000,0.000000,0.010870,0.000000,0.000000,0.000000,0.000000,0.010870,0.021739,0.010870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.010870,0.000000,0.000000,0.000000,0.000000,0.000000,0.010870,0.032609,0.032609,0.010870,...,0.000000,0.032609,0.010870,0.010870,0.021739,0.021739,0.000000,0.000000,0.010870,0.021739
1996,0.021739,0.000000,0.000000,0.010870,0.054348,0.010870,0.021739,0.010870,0.000000,0.000000,...,0.010870,0.000000,0.010870,0.021739,0.010870,0.000000,0.000000,0.043478,0.010870,0.010870
1997,0.010870,0.000000,0.010870,0.000000,0.000000,0.000000,0.000000,0.000000,0.054348,0.043478,...,0.010870,0.010870,0.000000,0.000000,0.010870,0.141304,0.000000,0.021739,0.032609,0.032609
1998,0.021739,0.032609,0.032609,0.010870,0.021739,0.010870,0.010870,0.032609,0.000000,0.032609,...,0.000000,0.032609,0.010870,0.000000,0.000000,0.000000,0.021739,0.021739,0.000000,0.010870


## Ridge regression ( équivalent à Kernel ridge régression avec noyau linéaire) 
On ajoute une colonne de biais 

In [6]:
X_0train_forRR=np.hstack((np.ones((X_0train.shape[0],1)),X_0train))
X_1train_forRR=np.hstack((np.ones((X_0train.shape[0],1)),X_0train))
X_2train_forRR=np.hstack((np.ones((X_0train.shape[0],1)),X_0train))

### Solver ridge Regression 
Même si les classes sont équilibrées je laisse la possibilité de modifier les poids. Enfin pas vraiment puisque le seul poids que je met quand on souhaite en mettre c'est le poid correspondant au poids des classes dans le set qui sera entraîné.
J'utilisais lstsq pour résoudre le système lineaire car plus stable si la matrice est mal conditionné/pas inversible mais cela prend beaucoup plus de temps que solve, ducoup je suis repassé à solve.

In [7]:
def Ridge_Regresssion(X_train,y_train,lbd,weight=False):
    n=y_train.shape[0]
    d=X_train.shape[1]
    if not(weight):
        A=(X_train.T.dot(X_train))/n+lbd*np.eye(d)
        B=X_train.T.dot(y_train)/n
        theta=solve(A,B)
        return theta
    else:
        w1=(y_train==1).mean()
        w0=1-w1
        w=np.where(y_train==1,w1,w0)
        W=np.diag(w)
        A=(X_train.T.dot(W.dot(X_train)))/n+lbd*np.eye(d)
        B=X_train.T.dot(W.dot(y_train))/n
        theta=solve(A,B)
        return theta


 ### Création de l'estimateur Ridge regression compatible avec l'API de scikit learn

In [8]:
class LinearRR(BaseEstimator,ClassifierMixin):
    def __init__(self,lbd=1,weight=False):
        self.lbd=lbd
        self.weight=weight
        
    def fit(self,X,y):
        self.classes_ = np.unique(y)
        
        self.theta=Ridge_Regresssion(X,y,self.lbd,self.weight)
        return self
    def decision_function(self,X):
        return X.dot(self.theta)
        
    def predict(self,X,y=None):
        scores=self.decision_function(X)
        if len(scores.shape) == 1:
            indices = (scores > 0).astype(np.int)
        else:
            indices = scores.argmax(axis=1)
        return self.classes_[indices]
   
    def get_params(self, deep=True):
    
        return {"lbd": self.lbd,"weight":self.weight}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

### Fonction permettant d'afficher les n meilleurs résultats avec les paramètres associées d'une grid search dans un tableau (dataframe)

In [9]:
def présentation_résultat(search,n):
    mask=search.cv_results_['rank_test_score']<=n
    params=list(compress(search.cv_results_['params'], list(mask)))
    mean_test_score=search.cv_results_['mean_test_score'][mask]
    a={}
    for i in range(mean_test_score.size):
        k=''
        for key, value in params[i].items():
            k+=" "+key+" "+str(value)
        a.update({k:mean_test_score[i]})
        sortedDict = sorted(a.items(), key=lambda x: x[1],reverse=True)
    l=[]
    for i in sortedDict:
        u=i[0].split(sep=' ')
        del(u[0])
        lp=[]
        for j in u[1::2]:
            lp.append(j)
        lp.append(i[1])
        l.append(lp)
    head=list(params[0].keys())+["mean_test_score"]

    return(pd.DataFrame(l,columns=head))
        
    
    
    
    

## Première recherche des hyperparamètres pour chaque train set 
Je crée un pipeline composé d'un transformer de données et de la ridge regression. Dans la manière dont j'ai défini la pipeline le transformer en question est *StandardScaler()* qui standardise les données ((X-mean)/std)). Attention cela standardise les données selon la mean et std du train set et donc lors d'une prédiction le test/validation set sera standardisé par rapport à la mean et std du training set. 
En soit la manière dont j'ai défini le tranformeur n'a pas d'importance car dans la gridSearch je teste d'autres transformeur à certain moment(mais fallait bien en mettre pour faire la pipeline), il y a même une option *"passthrough"* pour la première étape de la pipeline qui veut dire tout simplement passer à l'étape suivante et donc ne pas transfomrer mes données.


In [10]:
RR=Pipeline([("preprocess",StandardScaler()),("linearrr",LinearRR())])
print(RR)

hps0={"linearrr__lbd":[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"linearrr__weight":[False,True],
     "preprocess":["passthrough",StandardScaler(),RobustScaler(),MinMaxScaler()]}
searchLRR0= GridSearchCV(RR,hps0,scoring="accuracy",cv=5,verbose=1)
searchLRR0.fit(X_0train_forRR, Y_0train)

hps1={"linearrr__lbd":[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"linearrr__weight":[False,True],
     "preprocess":["passthrough",StandardScaler(),RobustScaler(),MinMaxScaler()]}
searchLRR1= GridSearchCV(RR,hps1,scoring="accuracy",cv=5,verbose=1)
searchLRR1.fit(X_1train_forRR, Y_1train)


hps2={"linearrr__lbd":[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10],"linearrr__weight":[False,True],
     "preprocess":["passthrough",StandardScaler(),RobustScaler(),MinMaxScaler()]}
searchLRR2= GridSearchCV(RR,hps2,scoring="accuracy",cv=5,verbose=1)
searchLRR2.fit(X_2train_forRR, Y_2train)


Pipeline(steps=[('preprocess', StandardScaler()), ('linearrr', LinearRR())])
Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed:    9.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed:    9.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed:    8.9s finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess', StandardScaler()),
                                       ('linearrr', LinearRR())]),
             param_grid={'linearrr__lbd': [1e-06, 1e-05, 0.0001, 0.001, 0.01,
                                           0.1, 1, 10],
                         'linearrr__weight': [False, True],
                         'preprocess': ['passthrough', StandardScaler(),
                                        RobustScaler(), MinMaxScaler()]},
             scoring='accuracy', verbose=1)

## Affichage des résulats 

In [11]:
présentation_résultat(searchLRR0,10)


,linearrr__lbd,linearrr__weight,preprocess,mean_test_score
0,1,True,StandardScaler(),0.5975
1,1,False,StandardScaler(),0.5945
2,0.1,True,StandardScaler(),0.5940
3,0.01,False,MinMaxScaler(),0.5930
4,0.1,False,StandardScaler(),0.5905
5,0.001,True,MinMaxScaler(),0.5890
6,0.01,False,StandardScaler(),0.5885
7,0.01,True,StandardScaler(),0.5870
8,0.001,True,StandardScaler(),0.5865
9,1e-06,True,StandardScaler(),0.5860


In [12]:
présentation_résultat(searchLRR1,10)

,linearrr__lbd,linearrr__weight,preprocess,mean_test_score
0,0.1,False,passthrough,0.5150
1,0.1,True,MinMaxScaler(),0.5150
2,10,True,StandardScaler(),0.5135
3,1,True,MinMaxScaler(),0.5130
4,10,True,MinMaxScaler(),0.5130
5,1,False,MinMaxScaler(),0.5125
6,0.01,True,passthrough,0.5125
7,1,True,RobustScaler(),0.5110
8,10,False,StandardScaler(),0.5110
9,0.001,False,passthrough,0.5090


In [13]:
présentation_résultat(searchLRR2,10)

,linearrr__lbd,linearrr__weight,preprocess,mean_test_score
0,0.01,True,passthrough,0.5015
1,0.1,True,passthrough,0.5015
2,1,False,passthrough,0.5015
3,1,True,passthrough,0.5015
4,10,False,passthrough,0.5015
5,10,True,passthrough,0.5015
6,10,True,MinMaxScaler(),0.5010
7,0.1,False,passthrough,0.5005
8,10,True,StandardScaler(),0.4995
9,0.001,False,passthrough,0.4980


### Recherche plus fine dans les zones d'intérêts déterminés à l'aide la search précedente

Ici je fais une RandomizedGrid search car j'ai beaucoup plus d'informations sur les zones d'intérêts où il faut chercher(notamment pour le lambda). Si je l'avais fait avant étant donné que j'avais aucun à priori sur la valeur de lambda la recherche au hasard aurait été plus difficile. 

In [14]:

hps0={"linearrr__lbd":uniform(loc=0.005,scale=5),"linearrr__weight":[False,True]}
searchLRR0= RandomizedSearchCV(RR,hps0,n_iter=500,scoring="accuracy",cv=5,verbose=1)
searchLRR0.fit(X_0train_forRR, Y_0train)

hps1={"linearrr__lbd":uniform(loc=0.05,scale=15),"linearrr__weight":[False,True],
     "preprocess":["passthrough",StandardScaler(),RobustScaler(),MinMaxScaler()]}
searchLRR1= RandomizedSearchCV(RR,hps1,n_iter=1000,scoring="accuracy",cv=5,verbose=1)
searchLRR1.fit(X_1train_forRR, Y_1train)


hps2={"linearrr__lbd":uniform(loc=0.005,scale=15),"linearrr__weight":[False,True],
     "preprocess":["passthrough"]}
searchLRR2= RandomizedSearchCV(RR,hps2,n_iter=500,scoring="accuracy",cv=5,verbose=1)
searchLRR2.fit(X_2train_forRR, Y_2train)


Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 

### Présentation des résultats 

In [ ]:
présentation_résultat(searchLRR0,10)


In [16]:
présentation_résultat(searchLRR1,10)

,linearrr__lbd,linearrr__weight,preprocess,mean_test_score
0,2.662814399357097,True,MinMaxScaler(),0.5165
1,1.63118211054454,True,MinMaxScaler(),0.5160
2,6.022893011897496,True,MinMaxScaler(),0.5155
3,5.034094985975762,True,MinMaxScaler(),0.5155
4,2.786563568420897,True,MinMaxScaler(),0.5155
5,5.591061637065799,True,MinMaxScaler(),0.5155
6,2.777275514245422,True,MinMaxScaler(),0.5155
7,5.419640410398476,True,MinMaxScaler(),0.5155
8,2.7804350268823796,True,MinMaxScaler(),0.5155
9,6.229126490300691,True,MinMaxScaler(),0.5150


In [17]:
présentation_résultat(searchLRR2,2)

,linearrr__lbd,linearrr__weight,preprocess,mean_test_score
0,0.061342756436065786,False,passthrough,0.5035
1,0.019400442629027844,True,passthrough,0.5025


### Recherche encore plus fine (sert pas trop à grand chose car on va garder ce modèle)

In [ ]:
hps0={"linearrr__lbd":uniform(loc=0.8,scale=1.2),"linearrr__weight":[False]}
searchLRR0= RandomizedSearchCV(RR,hps0,n_iter=100,scoring="accuracy",cv=5,verbose=1)
searchLRR0.fit(X_0train_forRR, Y_0train)

hps1={"linearrr__lbd":uniform(loc=0.01,scale=1.5),"linearrr__weight":[False],
     "preprocess":["passthrough"]}
searchLRR1= RandomizedSearchCV(RR,hps1,n_iter=100,scoring="accuracy",cv=5,verbose=1)
searchLRR1.fit(X_1train_forRR, Y_1train)


hps2={"linearrr__lbd":uniform(loc=0.005,scale=0.1),"linearrr__weight":[False],
     "preprocess":["passthrough"]}
searchLRR2= RandomizedSearchCV(RR,hps2,n_iter=100,scoring="accuracy",cv=5,verbose=1)
searchLRR2.fit(X_2train_forRR, Y_2train)


### Présentation des résultats 

In [ ]:
présentation_résultat(searchLRR0,10)


In [ ]:
présentation_résultat(searchLRR1,10)

In [ ]:
présentation_résultat(searchLRR2,10)

# Kernel Ridge regression 

### Crétion du kernel de manière efficiente voir ce lien pour comprendre https://stackoverflow.com/questions/47271662/what-is-the-fastest-way-to-compute-an-rbf-kernel-in-python 
Mais ce qu'il y a retenir c'est que $\| x-y \|^2=\|x\|^2 + \|y\|^2 -2x^Ty$ et d'utiliser à son avantage le broadcast de Numpy. Le ne.evaluate est là pour rendre l'opération plus rapide mais l'opération qui est faite et bien "exp(-g*(A+B-2*C))" Avec les A B C g correspondant. D'ailleurs la propriété précédente nous permet de manière générale de calculer la matrice $K(x_i,y_i)$ pour $y_1,\dots,y_k ~~x_1,\dots,x_n$  $y_i,x_i \in R^d$ La kernel matrice et un cas particulier où $n=k$ et $x_i=y_i$ pour tout $i$.
Cette fonction est contruite juste après la fonction qui construit la kernel matrice. Elle nous sera très utile en espérant que vous aviez déjà deviné pourquoi, c'est pour  calculer $\left(f(z_1),\dots,f(z_k)\right)$ oû $f(z_j)=\sum_{i=1}^{n}\alpha_i K(z_j,x_i)$ 


In [15]:
def rbf_kernel(X_train,gamma):
    X_norm = np.sum(X_train ** 2, axis = -1)
    K = ne.evaluate('exp(-g * (A + B - 2 * C))', {
        'A' : X_norm[:,None],
        'B' : X_norm[None,:],
        'C' : np.dot(X_train, X_train.T),
        'g' : gamma,
    })
    return K

In [16]:
def K_rbf_kernel(X_test,X_train,gamma):
   
    Xtt_norm = np.sum(X_test ** 2, axis = -1)
    Xtr_norm = np.sum(X_train ** 2, axis = -1)
    K = ne.evaluate('exp(-g * (A + B - 2 * C))', {
        'A' : Xtt_norm[:,None],
        'B' : Xtr_norm[None,:],
        'C' : np.dot(X_test, X_train.T),
        'g' : gamma,
    })
    return K
    

In [17]:
def poly_kernel(X,Y,degree,c0):
    return (X.dot(Y.T)+c0)**degree

In [18]:
def addbiais(X):
    return np.hstack((X,np.ones((X.shape[0],1))))
def addzeros(X):
    n,_=X.shape
    A=np.zeros((n+1,n+1))
    A[:n,:n]=X
    return(A)

    
    


### Création du solver Kernel ridge regresion

Comme vous l'avez remarqué je laisse la possibilité d'ajouter un paramètre de biais à l'estimateur ($f(x)=\sum_{i=1}^{n}K(xi,x)\,+b)$ mais qui n'est pas pénalisé!! car cela aurait aucun sens. Cela mène bien au système d'équation que je résouds dans la partie *bais* du solver qui suit, vérifier à la main si vous n'êtes pas convaincu

In [19]:
def Kernel_Ridge_Regression(X_train,y_train,lbd,weight,gamma,degree,c0,k,biais,kernel):
    if kernel=="rbf":
        K=rbf_kernel(X_train,gamma)
    elif kernel=="poly":
        K=poly_kernel(X_train,X_train,degree,c0)
    elif kernel=="spectrum":
        K=spectrum_kernel(X_train,k)
    elif kernel=="precomputed":
        K=X_train
    n=K.shape[0]
    w=weight
    if not(biais):
        if isinstance(weight,bool):
            A=(K+n*lbd*np.eye(n))
            alpha=solve(A,y_train,assume_a="sym")
            return alpha
        elif isinstance(weight,str):
                w1=(y_train==1).mean()
                w0=1-w1
                w=np.where(y_train==1,w1,w0)
        wi=(1/w)
        
        A=K+n*lbd*wi*np.eye(n)
        alpha=solve(A,y_train,assume_a="sym")
     
        return alpha
    else:
        
        Kb=addbiais(K)
        
        K0=addzeros(K)
    
        if isinstance(weight,bool):
            A=(Kb.T.dot(Kb)+lbd*n*K0)
            B=Kb.T.dot(y_train)
            alpha=solve(A,B,assume_a="sym")
            return alpha
        elif isinstance(weight,str):
                w1=(y_train==1).mean()
                w0=1-w1
                w=np.where(y_train==1,w1,w0)
        W=np.diag(w)
        A=(Kb.T.dot(W.dot(Kb))+lbd*n*K0)
        B=Kb.T.dot(W.dot(y_train),assume_a="sym")
        alpha=solve(A,B)
        return alpha
        
    
    

### Création de l'estimateur compatible avec l'Api de scikit 

In [20]:
class KernelRR(BaseEstimator,ClassifierMixin):
    def __init__(self,lbd=1,weight=False,gamma="auto",degree=2,c0=1,k=3,biais=False,kernel="rbf"):
        self.lbd=lbd
        self.weight=weight
        self.gamma=gamma
        self.degree=degree
        self.c0=c0
        self.k=k
        self.biais=biais
        self.kernel=kernel
    def fit(self,X,y):
        self.classes_ = np.unique(y)
        self.Xtr=X
        if isinstance(self.gamma,str) and self.kernel=="rbf":
            self.gamma=1/self.Xtr.shape[1]
        self.alpha=Kernel_Ridge_Regression(X,y,self.lbd,self.weight,self.gamma,self.degree,self.c0,self.k,self.biais,self.kernel)
        return self
    def decision_function(self,X):
        if self.kernel=="precomputed":
            return X.dot(self.alpha)
        if not(self.biais):
            if self.kernel=="rbf":
                return K_rbf_kernel(X,self.Xtr,self.gamma).dot(self.alpha) 
            elif self.kernel=="poly":
                return poly_kernel(X,self.Xtr,self.degree,self.c0).dot(self.alpha) 
            elif self.kernel=="spectrum":
                return K_spectrum_kernel(X,self.Xtr,self.k).dot(self.alpha) 
        else:
            if self.kernel=="rbf":
                return addbiais(K_rbf_kernel(X,self.Xtr,self.gamma)).dot(self.alpha)
            elif self.kernel=="poly":
                return addbiais(poly_kernel(X,self.Xtr,self.degree,self.c0)).dot(self.alpha)
            elif self.kernel=="spectrum":
                return addbiais(K_spectrum_kernel(X,self.Xtr,self.k)).dot(self.alpha)

    def predict(self,X,y=None):
        scores=self.decision_function(X)
        if len(scores.shape) == 1:
            indices = (scores > 0).astype(np.int)
        else:
            indices = scores.argmax(axis=1)
        return self.classes_[indices]
   
    def get_params(self, deep=True):
    
        return {"lbd": self.lbd,"weight":self.weight,"gamma":self.gamma,"degree":self.degree,"c0":self.c0,"k":self.k,
                "biais":self.biais,"kernel":self.kernel}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

### Recherche des hyperparamètres pour KRR avec noyau gaussien 
Pour la transformation des données ici vu qu'on va ensuite appliqué un kernel qui va rédéfinir les distance entre le points la seul transformation qui à un sens est celle où l'on standardise les données car elle conserve les rapports de distance(et donc pas affecté le kernel). De plus point très important, le paramètre *gamma* du noyau et *lambda* de la régularisation peuvent ne pas jouer de pair dans la performance de l'estimateur, il est donc difficile d'optimiser à la fois *lambda* et *gamma*. Effectivement il est possible que pour un *gamma* haut on ait besoin d'un *lambda* bas pour performer et inversement et comme on ne veut pas se perdre dans différents chemins, il semble logique de fixer *gamma* puis d'optimer *lambda*. Cela tombe bien puisque sans rentrer dans les détails mais lorsque que les données sont standardisées un paramètre de *gamma* naturel est 1/(num_features).

In [21]:
KRR=make_pipeline(StandardScaler(),KernelRR(kernel="rbf"))
print(KRR)

hps0={'kernelrr__lbd':[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10]}
searchKRR0= GridSearchCV(KRR,hps0,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKRR0.fit(X_0train, Y_0train)

hps1={"kernelrr__lbd":[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10]}
searchKRR1= GridSearchCV(KRR,hps1,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKRR1.fit(X_1train, Y_1train)


hps2={"kernelrr__lbd":[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10]}
searchKRR2= GridSearchCV(KRR,hps2,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKRR2.fit(X_2train, Y_2train)



Pipeline(steps=[('standardscaler', StandardScaler()), ('kernelrr', KernelRR())])
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    9.9s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    7.5s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    6.1s finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('kernelrr', KernelRR())]),
             n_jobs=-1,
             param_grid={'kernelrr__lbd': [1e-06, 1e-05, 0.0001, 0.001, 0.01,
                                           0.1, 1, 10]},
             scoring='accuracy', verbose=1)

### Présentation des Résultats 
On remarque pour le training set 1 on a 10% en plus 20% en plus!!! pour le training set 2 et rien ne change pour le training set 0 par rapport au modèle linéraire. A Approfondir

In [22]:
présentation_résultat(searchKRR0,10)


,kernelrr__lbd,mean_test_score
0,0.001,0.5995
1,0.01,0.5910
2,0.0001,0.5885
3,1e-05,0.5830
4,1e-06,0.5825
5,0.1,0.5480
6,1,0.5260
7,10,0.5225


In [23]:
présentation_résultat(searchKRR1,10)

,kernelrr__lbd,mean_test_score
0,0.001,0.6055
1,0.0001,0.5990
2,0.01,0.5915
3,1e-05,0.5865
4,1e-06,0.5820
5,0.1,0.5530
6,10,0.5255
7,1,0.5255


In [24]:
présentation_résultat(searchKRR2,10)

,kernelrr__lbd,mean_test_score
0,0.001,0.7100
1,0.01,0.6950
2,0.0001,0.6895
3,1e-06,0.6765
4,1e-05,0.6765
5,0.1,0.6685
6,10,0.6545
7,1,0.6530


In [69]:
KRR=make_pipeline(StandardScaler(),KernelRR(weight=False,gamma="auto",kernel="rbf"))
print(KRR)

hps0={"kernelrr__lbd":uniform(loc=10**-4,scale=5*10**-2)}
searchKRR0= RandomizedSearchCV(KRR,hps0,n_iter=500,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKRR0.fit(X_0train, Y_0train)

hps1={"kernelrr__lbd":uniform(loc=5*10**-5,scale=5*10**-2)}
searchKRR1=RandomizedSearchCV(KRR,hps1,n_iter=500,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKRR1.fit(X_1train, Y_1train)


hps2={"kernelrr__lbd":uniform(loc=5*10**-5,scale=5*10**-2)}
searchKRR2= RandomizedSearchCV(KRR,hps2,n_iter=100,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKRR2.fit(X_2train, Y_2train)


Pipeline(steps=[('standardscaler', StandardScaler()), ('kernelrr', KernelRR())])
Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   32.7s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 2500 out of 2500 | elapsed:  7.0min finished


Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   37.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.4min


KeyboardInterrupt: 

### Présentation des résultats

In [ ]:
présentation_résultat(searchKRR0,10)


In [ ]:
présentation_résultat(searchKRR1,10)

In [ ]:
présentation_résultat(searchKRR2,10)

### Polynomial kernel

# Logistic Regression 

In [25]:
def sigmoid(v):
    return 1/(1+np.exp(-v))
def log_loss(v):
    return np.log(1+np.exp(-v))
def cross_loss(y,v):
    return -(y*np.log(v)+(1-y)*np.log(1-v))
from scipy.special import expit

## Solver Kernel Logistic Regression 

Le critère d'arrêt que j'ai mis c'est quand la différence deux itétéres consécutifs a une norme inférieure à $\epsilon$. Ici aussi je laisse la possibilité d'un biais.

In [26]:
def IRLS(X_train,y_train,lbd,ga,degree,c0,k,bs,ker,n_iter,eps=10**-6,method='slow'):
    n=y_train.shape[0]
  
    if ker=="rbf":
        K=rbf_kernel(X_train,ga)
    elif ker=="poly":
        K=poly_kernel(X_train,X_train,degree,c0)
    elif ker=="spectrum":
        K=spectrum_kernel(X_train,k)
    elif ker=="precomputed":
        K=X_train
    #alpha=Kernel_Ridge_Regression(K,y_train,lbd,False,1,bs,"precomputed")
    #alpha=np.zeros(n)
    #l=[]
  
    if bs :
        Kb=addbiais(K)
        K0=addzeros(K)
        alpha=np.zeros(n+1)
    else:
        alpha=np.zeros(n)
    for i in range(n_iter):
   
        alpha_old=alpha
       
        if bs:
            m=Kb.dot(alpha)
            #l.append(log_loss(y_train*m).mean()+lbd*alpha[:-1].dot(K.dot(alpha[:-1])))
        
        else:
            m=K.dot(alpha)
            #l.append(log_loss(y_train*m).mean()+lbd*alpha.dot(m))
        
        
        p=sigmoid(m)
       
        weight=p*(1-p)
       
        weight=np.where(weight<0.000001,0.000001,weight)
       
   
        u=np.where(sigmoid(y_train*m)<0.000001,0.000001,sigmoid(y_train*m))
        z = m + y_train/u
    
        if not(bs):
        
            S = np.diag(weight**-1)
            A=(K+2*lbd*n*S)
            alpha=solve(A,z,assume_a="sym")
            
            #print(np.linalg.norm(alpha_old-alpha))
            
            if np.linalg.norm(alpha_old-alpha)<eps:
                break
        else:
            S = np.diag(weight)
            A=(Kb.T.dot(S.dot(Kb))+2*lbd*n*K0)
            B=Kb.T.dot(S.dot(z))
            if method=="slow":
                alpha=lstsq(A,B)[0]
            else:
                alpha=solve(A,B,assume_a="sym")
            #print(np.linalg.norm(alpha_old-alpha))
            if np.linalg.norm(alpha_old-alpha)<eps:
                break
                
       
    return alpha #,l
        
#with np.printoptions(threshold=np.inf):

### Je construit l'estimateur compatible avec l'API de scikit 

In [27]:
class KernelLR(BaseEstimator,ClassifierMixin):
    def __init__(self,lbd=1,gamma='auto',degree=2,c0=1,k=3,biais=False,kernel="rbf",n_iter=15,method="slow"):
        self.lbd=lbd
        self.gamma=gamma
        self.degree=degree
        self.c0=c0
        self.k=k
        self.biais=biais
        self.kernel=kernel
        self.n_iter=n_iter
        self.method=method
    def fit(self,X,y):
        self.classes_ = np.unique(y)
        self.Xtr=X
        if isinstance(self.gamma,str) and self.kernel=="rbf":
            self.gamma=1/self.Xtr.shape[1]
        self.alpha=IRLS(X,y,self.lbd,self.gamma,self.degree,self.c0,self.k,self.biais,self.kernel,self.n_iter,method=self.method)
        return self
    def decision_function(self,X):
        if not(self.biais):
            if self.kernel=="precomputed":
                return X.dot(self.alpha)
            if self.kernel=="rbf":
                return K_rbf_kernel(X,self.Xtr,self.gamma).dot(self.alpha) 
            elif self.kernel=="poly":
                return poly_kernel(X,self.Xtr,self.degree,self.c0).dot(self.alpha) 
            elif self.kernel=="spectrum":
                return K_spectrum_kernel(X,self.Xtr,self.k).dot(self.alpha) 
        else:
            if self.kernel=="rbf":
                return addbiais(K_rbf_kernel(X,self.Xtr,self.gamma)).dot(self.alpha)
            elif self.kernel=="poly":
                return addbiais(poly_kernel(X,self.Xtr,self.degree,self.c0)).dot(self.alpha)
            elif self.kernel=="spectrum":
                return addbiais(K_spectrum_kernel(X,self.Xtr,self.k)).dot(self.alpha)
        

    def predict(self,X,y=None):
        scores=self.decision_function(X)
        if len(scores.shape) == 1:
            indices = (scores > 0).astype(np.int)
        else:
            indices = scores.argmax(axis=1)
        return self.classes_[indices]
    def predict_proba_(self,X,y=None):
        p=sigmoid(self.decision_function(X)).reshape(-1,1)
        return hstack((p,1-p))
    def get_params(self, deep=True):
    
        return {"lbd": self.lbd,"gamma":self.gamma,"degree":self.degree,"c0":self.c0,"k":self.k,"biais":self.biais,
                "kernel":self.kernel,"n_iter":self.n_iter,"method":self.method}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

### Recherche des Hyperparamètres 

In [28]:
KLR=make_pipeline(StandardScaler(),KernelLR())
print(KLR)

hps0={'kernellr__lbd':[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10]}
searchKLR0= GridSearchCV(KLR,hps0,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKLR0.fit(X_0train, Y_0train)

hps1={'kernellr__lbd':[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10]}
searchKLR1= GridSearchCV(KLR,hps1,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKLR1.fit(X_1train, Y_1train)


hps2={'kernellr__lbd':[10**-6,10**-5,10**-4,10**-3,10**-2,10**-1,1,10]}
searchKLR2= GridSearchCV(KLR,hps2,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKLR2.fit(X_2train, Y_2train)


Pipeline(steps=[('standardscaler', StandardScaler()), ('kernellr', KernelLR())])
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   21.1s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   20.5s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   23.1s finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('kernellr', KernelLR())]),
             n_jobs=-1,
             param_grid={'kernellr__lbd': [1e-06, 1e-05, 0.0001, 0.001, 0.01,
                                           0.1, 1, 10]},
             scoring='accuracy', verbose=1)

### Présentation des résultats 

In [29]:
présentation_résultat(searchKLR0,10)

,kernellr__lbd,mean_test_score
0,0.0001,0.6010
1,1e-05,0.5890
2,0.001,0.5890
3,1e-06,0.5845
4,0.01,0.5515
5,0.1,0.5250
6,10,0.5225
7,1,0.5225


In [30]:
présentation_résultat(searchKLR1,10)

,kernellr__lbd,mean_test_score
0,0.0001,0.6055
1,1e-06,0.5970
2,1e-05,0.5965
3,0.001,0.5960
4,0.01,0.5515
5,0.1,0.5265
6,10,0.5255
7,1,0.5250


In [31]:
présentation_résultat(searchKLR2,10)

,kernellr__lbd,mean_test_score
0,0.0001,0.7110
1,1e-05,0.6940
2,0.001,0.6940
3,1e-06,0.6905
4,0.01,0.6670
5,10,0.6545
6,1,0.6535
7,0.1,0.6530


In [ ]:
hps0={"kernellr__lbd":uniform(loc=5*10**-6,scale=5*10**-3)}
searchKLR0= RandomizedSearchCV(KLR,hps0,n_iter=500,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKLR0.fit(X_0train, Y_0train)

hps1={"kernellr__lbd":uniform(loc=5*10**-7,scale=5*10**-3)}
searchKLR1=RandomizedSearchCV(KLR,hps1,n_iter=500,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKLR1.fit(X_1train, Y_1train)


hps2={"kernellr__lbd":uniform(loc=5*10**-7,scale=5*10**-3)}
searchKLR2= RandomizedSearchCV(KLR,hps2,n_iter=500,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKLR2.fit(X_2train, Y_2train)

In [ ]:
présentation_résultat(searchKLR0,10)

In [ ]:
présentation_résultat(searchKLR1,10)

In [ ]:
présentation_résultat(searchKLR2,10)

# SVM 
J'utilise cvxopt pour résoudre le QP

In [33]:
def SVM(X_train,y_train,C,gamma,degree,c0,k,kernel):
    n=y_train.shape[0]
    if kernel=="rbf":
        K=rbf_kernel(X_train,gamma)
    elif kernel=="poly":
        K=poly_kernel(X_train,X_train,degree,c0)
    elif kernel=="spectrum":
        K=spectrum_kernel(X_train,k)
    elif kernel=="precomputed":
        K=X_train
    P=matrix(K,tc='d')
    q=matrix(-y_train,tc='d')
    g1=np.diag(y_train)
    G=matrix(np.vstack((g1,-g1)),tc='d')
    h=matrix(np.hstack((np.repeat(C,n),np.zeros(n))),tc='d')
    solvers.options['show_progress'] = False
    sol=solvers.qp(P,q,G,h)
    return np.array(sol['x']).reshape(-1,)

### Création de l'estimateur compatible avec l'API de scikit 

Si vous remarquez bien j'utilise tout alpha pour définir ma fonction f(x) alors que je pourrai garder seulement les alpha_i telle que alpha_i plus grand > threshold (car le solver nous fera jamais arrivé exatement à zéro quand alpha_i doit être égale à zéro) car comme on le sait une solution optimale de SVM est un vecteur sparse. Je ne l'ai pas fait car j'ai eu du mal à définir le threshold, on trouve une trace de ma tentative dans le code de l'estimateur

In [34]:
class KernelSVM(BaseEstimator,ClassifierMixin):
    def __init__(self,C=1,gamma='auto',degree="2",c0=1,k=3,kernel="rbf"):
        self.C=C
        self.gamma=gamma
        self.degree=degree
        self.c0=c0
        self.k=k
        self.kernel=kernel
    def fit(self,X,y):
        self.classes_ = np.unique(y)
        self.Xtr=X
        if isinstance(self.gamma,str) and self.kernel=="rbf":
            self.gamma=1/self.Xtr.shape[1]
        self.alpha=SVM(X,y,self.C,self.gamma,self.degree,self.c0,self.k,self.kernel)
        #idx=self.alpha>10**-5
        #self.Xtr=self.Xtr[idx]
        #self.alpha=self.alpha[idx]
        return self
    def decision_function(self,X):
        if self.kernel=="precomputed":
            return X.dot(self.alpha)
        elif self.kernel=="rbf":
            return K_rbf_kernel(X,self.Xtr,self.gamma).dot(self.alpha) 
        elif self.kernel=="poly":
            return poly_kernel(X,self.Xtr,self.degree,self.c0).dot(self.alpha) 
        elif self.kernel=="spectrum":
            return K_spectrum_kernel(X,self.Xtr,self.k).dot(self.alpha) 
            
           
    def predict(self,X,y=None):
        scores=self.decision_function(X)
        if len(scores.shape) == 1:
            indices = (scores > 0).astype(np.int)
        else:
            indices = scores.argmax(axis=1)
        return self.classes_[indices]
    def get_params(self, deep=True):
    
        return {"C": self.C,"gamma":self.gamma,"degree":self.degree,"c0":self.c0,"k":self.k,"kernel":self.kernel}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

In [35]:
KSVM=make_pipeline(StandardScaler(),KernelSVM())
print(KSVM)

hps0={'kernelsvm__C':[10**-2,10**-1,1,10,50,100,500,1000]}
searchKSVM0= GridSearchCV(KSVM,hps0,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKSVM0.fit(X_0train, Y_0train)

hps1={'kernelsvm__C':[10**-2,10**-1,1,10,50,100,500,1000]}
searchKSVM1= GridSearchCV(KSVM,hps1,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKSVM1.fit(X_1train, Y_1train)


hps2={'kernelsvm__C':[10**-2,10**-1,1,10,50,100,500,1000]}
searchKSVM2= GridSearchCV(KSVM,hps2,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKSVM2.fit(X_2train, Y_2train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kernelsvm', KernelSVM())])
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  2.7min finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  2.7min finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  3.1min finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('kernelsvm', KernelSVM())]),
             n_jobs=-1,
             param_grid={'kernelsvm__C': [0.01, 0.1, 1, 10, 50, 100, 500,
                                          1000]},
             scoring='accuracy', verbose=1)

In [36]:
présentation_résultat(searchKSVM0,10)

,kernelsvm__C,mean_test_score
0,1,0.5940
1,50,0.5845
2,100,0.5845
3,500,0.5845
4,1000,0.5845
5,10,0.5835
6,0.1,0.5670
7,0.01,0.5225


In [37]:
présentation_résultat(searchKSVM1,10)

,kernelsvm__C,mean_test_score
0,1,0.6030
1,100,0.5865
2,1000,0.5865
3,10,0.5860
4,50,0.5860
5,500,0.5860
6,0.01,0.5250
7,0.1,0.5210


In [38]:
présentation_résultat(searchKSVM2,10)

,kernelsvm__C,mean_test_score
0,1,0.7040
1,0.1,0.6845
2,10,0.6845
3,50,0.6840
4,100,0.6840
5,500,0.6840
6,1000,0.6840
7,0.01,0.6535


In [ ]:

hps0={"kernelsvm__C":uniform(loc=0.5,scale=5)}
searchKSVM0= RandomizedSearchCV(KSVM,hps0,n_iter=100,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKSVM0.fit(X_0train, Y_0train)

hps1={"kernelsvm__C":uniform(loc=0.5,scale=5)}
searchKSVM1= RandomizedSearchCV(KSVM,hps1,n_iter=100,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKSVM1.fit(X_1train, Y_1train)

hps2={"kernelsvm__C":uniform(loc=0.1,scale=3)}
searchKSVM2= RandomizedSearchCV(KSVM,hps2,n_iter=100,scoring="accuracy",cv=5,verbose=1,n_jobs=-1)
searchKSVM2.fit(X_2train, Y_2train)

In [ ]:
présentation_résultat(searchKSVM0,10)

In [ ]:
présentation_résultat(searchKSVM1,10)

In [ ]:
présentation_résultat(searchKSVM2,10)

In [ ]:
KLR0=make_pipeline(StandardScaler(),KernelLR(lbd=0.00018))
KLR1=make_pipeline(StandardScaler(),KernelLR(lbd=0.00016))
KLR2=make_pipeline(StandardScaler(),KernelLR(lbd=5.89*(10**-5)))

X_0test=pd.read_csv("Data/Xte0_mat100.csv",sep=' ',header=None).to_numpy()
X_1test=pd.read_csv("Data/Xte1_mat100.csv",sep=' ',header=None).to_numpy()
X_2test=pd.read_csv("Data/Xte2_mat100.csv",sep=' ',header=None).to_numpy()


print(X_1train.shape)

In [211]:
def csv_file(models): #models is a list of 3 models
    Y_test=np.empty(0)
    for X_train, Y_train, X_test, model  in zip([X_0train,X_1train,X_2train], [Y_0train,Y_1train,Y_2train], [X_0test,X_1test,X_2test], models):
        model.fit(X_train, Y_train)
        Y_pred=model.predict(X_test)
        Y_test=np.concatenate((Y_test,np.where(Y_pred==-1,0,Y_pred)), axis=0)
    
    Y_test=Y_test.reshape(len(Y_test),1)
    with np.printoptions(threshold=np.inf):
        print(Y_test.shape)
    ids=np.arange(Y_test.shape[0])
    ids=ids.reshape(len(ids),1)
    
    df=pd.DataFrame(data=np.concatenate((ids,Y_test), axis=1), columns=['Id','Bound'],dtype=np.int)
    
    return df.to_csv('kernel_spectrum_try2.csv', index = False, header=True)

In [ ]:
csv_file([KLR0,KLR1,KLR2])

# String kernel 


In [39]:
import time 
from tqdm import tqdm
from scipy.stats import rankdata
from scipy import sparse

In [40]:
X_0train=pd.read_csv('"/Users/elliotmuller/Documents/M2/Kernel Methods/Data Challenge/Xtr0.csv', sep=',')['seq']
X_1train=pd.read_csv('Data/Xtr1.csv', sep=',')['seq']
X_2train=pd.read_csv('Data/Xtr2.csv', sep=',')['seq']

X_0test=pd.read_csv('Data/Xte0.csv', sep=',')['seq']
X_1test=pd.read_csv('Data/Xte1.csv', sep=',')['seq']
X_2test=pd.read_csv('Data/Xte2.csv', sep=',')['seq']

Y_0=pd.read_csv("Data/Ytr0.csv",sep=',')
Y_1=pd.read_csv("Data/Ytr1.csv",sep=',')
Y_2=pd.read_csv("Data/Ytr2.csv",sep=',')

Y_0train=np.where(Y_0["Bound"]==0,-1,Y_0["Bound"])
Y_1train=np.where(Y_1["Bound"]==0,-1,Y_1["Bound"])
Y_2train=np.where(Y_2["Bound"]==0,-1,Y_2["Bound"])


FileNotFoundError: [Errno 2] No such file or directory: 'Data/Xtr0.csv'

In [133]:
def K_spectrum_kernel(X,Y,k,alphabet="ACGT"):
    voc=product(alphabet, repeat=k)
    voc=[''.join(elt) for elt in voc]
    phi_X=np.vstack(X.apply(lambda x: [x[i:i+k] for i in range(0, len(x)-k+1)]).apply(lambda x: np.array([x.count(v) for v in voc])).to_numpy())
    phi_Y=np.vstack(Y.apply(lambda x: [x[i:i+k] for i in range(0, len(x)-k+1)]).apply(lambda x: np.array([x.count(v) for v in voc])).to_numpy())
    return phi_X.dot(phi_Y.T)
def spectrum_kernel(X,k,alphabet="ACGT"):
    voc=product(alphabet, repeat=k)
    voc=[''.join(elt) for elt in voc]
    phi_X=np.vstack(X.apply(lambda x: [x[i:i+k] for i in range(0, len(x)-k+1)]).apply(lambda x: np.array([x.count(v) for v in voc])).to_numpy())
   
    return phi_X.dot(phi_X.T)
    


In [ ]:
print(spectrum_kernel(X_0train,8))

 79%|███████▉  | 51794/65536 [00:00<00:00, 121908.90it/s]

In [173]:
KSVM=KernelSVM(kernel="precomputed")
print(KSVM)

hps0={'C':[10**-2,10**-1,1,10,50,100,500,1000],'k':[2,3,4,5,6,7,8]}
searchKSVM0= Cross_val_spectrum(X_0train,Y_0train,KSVM,hps0)


hps1={'C':[10**-2,10**-1,1,10,50,100,500,1000],'k':[2,3,4,5,6,7,8]}
searchKSVM1= Cross_val_spectrum(X_1train,Y_1train,KSVM,hps1)



hps2={'C':[10**-2,10**-1,1,10,50,100,500,1000],'k':[2,3,4,5,6,7,8]}
searchKSVM2= Cross_val_spectrum(X_2train,Y_2train,KSVM,hps2)


 14%|█▍        | 1/7 [00:00<00:00,  6.35it/s]

KernelSVM(kernel='precomputed')


100%|██████████| 7/7 [06:14<00:00, 53.51s/it] 
56it [13:11, 14.13s/it]
100%|██████████| 7/7 [06:13<00:00, 53.42s/it] 
56it [35:18, 37.83s/it]
100%|██████████| 7/7 [06:59<00:00, 59.88s/it] 
56it [15:13, 16.31s/it]


In [183]:
présentation_résultat2(searchKSVM0,30)

,C,k,mean_test_score
0,0.01,8,0.6455
1,0.01,7,0.6450
2,0.1,8,0.6405
3,1,8,0.6405
4,10,8,0.6405
5,50,8,0.6405
6,100,8,0.6405
7,500,8,0.6405
8,1000,8,0.6405
9,0.01,6,0.6380


In [185]:
présentation_résultat2(searchKSVM1,40)

,C,k,mean_test_score
0,0.01,5,0.6435
1,0.01,6,0.6380
2,0.01,7,0.6295
3,0.1,7,0.6200
4,1,7,0.6200
5,10,7,0.6200
6,50,7,0.6200
7,100,7,0.6200
8,500,7,0.6200
9,1000,7,0.6200


In [186]:
présentation_résultat2(searchKSVM2,40)

,C,k,mean_test_score
0,0.01,7,0.7415
1,0.1,7,0.7335
2,1,7,0.7310
3,10,7,0.7310
4,50,7,0.7310
5,100,7,0.7310
6,500,7,0.7310
7,1000,7,0.7310
8,0.01,6,0.7290
9,0.01,8,0.7275


In [200]:
hps0={'C':uniform(loc=0.005,scale=0.5),'k':[8]}
searchKSVM0= Randomized_Cross_val_spectrum(X_0train,Y_0train,KSVM,hps0,50)


hps1={'C':uniform(loc=0.005,scale=0.5),'k':[5]}
searchKSVM1= Randomized_Cross_val_spectrum(X_1train,Y_1train,KSVM,hps1,50)



hps2={'C':uniform(loc=0.005,scale=0.5),'k':[7]}
searchKSVM2= Randomized_Cross_val_spectrum(X_2train,Y_2train,KSVM,hps2,50)


100%|██████████| 1/1 [06:23<00:00, 383.41s/it]
0it [00:00, ?it/s]

{'C': array([0.46666505, 0.26907594, 0.12523631, 0.45466107, 0.26347195,
       0.19420895, 0.18226131, 0.49544762, 0.01318606, 0.41101619,
       0.45164923, 0.24666404, 0.05044877, 0.3151696 , 0.05091285,
       0.42096472, 0.11227579, 0.2445298 , 0.30944494, 0.40553232,
       0.49069645, 0.41870792, 0.4170422 , 0.41755719, 0.4080609 ,
       0.05694352, 0.21573527, 0.01444699, 0.3645368 , 0.22691003,
       0.21797331, 0.07831073, 0.01927725, 0.21226899, 0.43667885,
       0.02985245, 0.0565717 , 0.19886476, 0.11112374, 0.30510266,
       0.08007239, 0.47218621, 0.35868019, 0.24887759, 0.29805176,
       0.25165495, 0.39362325, 0.45773882, 0.07267951, 0.05861149]), 'k': array([8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 8, 8, 8, 8])}


50it [09:22, 11.24s/it]
100%|██████████| 1/1 [00:05<00:00,  5.15s/it]
0it [00:00, ?it/s]

{'C': array([0.43176172, 0.26195195, 0.24518519, 0.41727215, 0.09133693,
       0.34584573, 0.4296129 , 0.27213185, 0.4455721 , 0.34425933,
       0.12402668, 0.38649431, 0.04172012, 0.40968717, 0.14548111,
       0.08202455, 0.48124638, 0.17726664, 0.41057927, 0.1769999 ,
       0.34431211, 0.04408077, 0.20784162, 0.39786877, 0.48186417,
       0.38823755, 0.42120227, 0.47118595, 0.06240904, 0.23616567,
       0.18784333, 0.23993454, 0.12151771, 0.49477509, 0.18250762,
       0.28094271, 0.20887174, 0.24207455, 0.40490859, 0.08339908,
       0.12515863, 0.06248199, 0.24669478, 0.26764686, 0.07934126,
       0.16136374, 0.1944452 , 0.30599783, 0.19357854, 0.08016435]), 'k': array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5])}


50it [10:43, 12.88s/it]
100%|██████████| 1/1 [01:19<00:00, 79.43s/it]
0it [00:00, ?it/s]

{'C': array([0.24811796, 0.19046197, 0.02332814, 0.37084938, 0.28486939,
       0.43829491, 0.26565659, 0.17937156, 0.06996736, 0.3658085 ,
       0.28177865, 0.06674257, 0.28349039, 0.25354877, 0.400276  ,
       0.49915773, 0.46203365, 0.43082258, 0.01423607, 0.42812112,
       0.4607758 , 0.38853215, 0.40922324, 0.25191396, 0.3117775 ,
       0.31132515, 0.39262195, 0.23339819, 0.3925347 , 0.1423535 ,
       0.1797383 , 0.40227858, 0.41858376, 0.05750593, 0.05113303,
       0.06096535, 0.29375129, 0.09998449, 0.32552638, 0.15315188,
       0.24206569, 0.41775379, 0.47600435, 0.43112248, 0.17612762,
       0.25724631, 0.27676258, 0.0929287 , 0.11508991, 0.18911116]), 'k': array([7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7])}


50it [18:04, 21.69s/it]


In [203]:
présentation_résultat2(searchKSVM0,30)

,C,k,mean_test_score
0,0.050448771307686095,8,0.6435
1,0.05091285130337835,8,0.6430
2,0.02985245022581617,8,0.6425
3,0.013186059645409972,8,0.6410
4,0.0569435202737617,8,0.6410
5,0.014446992786455778,8,0.6410
6,0.056571703359897856,8,0.6410
7,0.05861148630114598,8,0.6410


In [207]:
présentation_résultat2(searchKSVM1,30)

,C,k,mean_test_score
0,0.041720115086265545,5,0.6240
1,0.0440807747696007,5,0.6235
2,0.06240904239977856,5,0.6205
3,0.062481994426005764,5,0.6200
4,0.08202455387710322,5,0.6155
5,0.12151770942350837,5,0.6155
6,0.1613637411943103,5,0.6145
7,0.17726663919630986,5,0.6145
8,0.17699989754103973,5,0.6145
9,0.08339908141150759,5,0.6145


In [208]:
présentation_résultat2(searchKSVM2,30)

,C,k,mean_test_score
0,0.014236067163357343,7,0.7410
1,0.02332813750240192,7,0.7350
2,0.0699673590736214,7,0.7345
3,0.06096535102182892,7,0.7345
4,0.05113303466458518,7,0.7340
5,0.06674257022952479,7,0.7340
6,0.057505934607167124,7,0.7340
7,0.0999844942020891,7,0.7335
8,0.09292869707770801,7,0.7330
9,0.1150899081854529,7,0.7330


In [213]:
model0=KernelSVM(C=0.01,k=8,kernel="spectrum")
model1=KernelSVM(C=0.01,k=5,kernel="spectrum")
model2=KernelSVM(C=0.01,k=7,kernel="spectrum")

In [216]:
csv_file([model0,model1,model2])

(3000, 1)


In [172]:
def Cross_val_spectrum(X_train,Y_train,model,hps,cv=5):
    CV=StratifiedKFold(cv)
    dic_K={k:spectrum_kernel(X_train,k) for k in tqdm(hps["k"])}
    list_hp=[]
    list_val_score=[]
    idx_k=list(hps.keys()).index("k")
    
    for i in tqdm(product(*hps.values())):
        
        dic_hp={keys:values for keys,values in zip(hps.keys(),i)}
        list_hp.append(dic_hp)
        
        model.set_params(**dic_hp)
        acc_mean=0
        for train_idx,val_idx in CV.split(X_train,Y_train):
            K=dic_K[i[idx_k]]
            model.fit(K[train_idx][:,train_idx],Y_train[train_idx])
            Y_pred=model.predict(K[val_idx][:,train_idx])
            acc_mean+=accuracy_score(Y_train[val_idx],Y_pred)
        list_val_score.append(acc_mean/cv)
    return {"params":list_hp,"mean_test_score":np.array(list_val_score),"rank_test_score":rankdata(list_val_score)}
            
            
    
        

In [198]:
def Randomized_Cross_val_spectrum(X_train,Y_train,model,hps,n_iter,cv=5):
    CV=StratifiedKFold(cv)
    dic_K={k:spectrum_kernel(X_train,k) for k in tqdm(hps["k"])}
    list_hp=[]
    list_val_score=[]
    idx_k=list(hps.keys()).index("k")
    for keys,values in hps.items():
        if isinstance(values,scipy.stats._distn_infrastructure.rv_frozen):
            hps[keys]=values.rvs(size=n_iter)
        else:
            hps[keys]=np.random.choice(values,n_iter)
    print(hps)
    for i in tqdm(zip(*hps.values())):
        
        dic_hp={keys:values for keys,values in zip(hps.keys(),i)}
        list_hp.append(dic_hp)
        
        model.set_params(**dic_hp)
        acc_mean=0
        for train_idx,val_idx in CV.split(X_train,Y_train):
            K=dic_K[i[idx_k]]
            model.fit(K[train_idx][:,train_idx],Y_train[train_idx])
            Y_pred=model.predict(K[val_idx][:,train_idx])
            acc_mean+=accuracy_score(Y_train[val_idx],Y_pred)
        list_val_score.append(acc_mean/cv)
    return {"params":list_hp,"mean_test_score":np.array(list_val_score),"rank_test_score":rankdata(list_val_score)}
            
            

In [199]:
hps={'C':uniform(loc=0.5,scale=2),'k':[3]}
CV=StratifiedKFold(5)
list_hp=[]
n_iter=10
list_val_score=[]
dic_K={k:np.arange(k) for k in hps["k"]}
idx_k=list(hps.keys()).index("k")
for keys,values in hps.items():
    if isinstance(values,scipy.stats._distn_infrastructure.rv_frozen):
        hps[keys]=values.rvs(size=n_iter)
    else:
        hps[keys]=np.random.choice(values,n_iter)
print(hps)

for i in zip(*hps.values()):
    dic_hp={keys:values for keys,values in zip(hps.keys(),i)}
    KSVM.set_params(**dic_hp)
    print(dic_hp)
    for train_idx,test_idx in CV.split(X_0train,Y_0train):
        x=5
       
   
    

{'C': array([2.29109288, 1.28619423, 0.54370751, 1.51972859, 1.29469697,
       1.70426481, 2.27696594, 0.52956819, 1.45780128, 1.24405329]), 'k': array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3])}
{'C': 2.2910928819369967, 'k': 3}
{'C': 1.2861942342546977, 'k': 3}
{'C': 0.5437075121323791, 'k': 3}
{'C': 1.5197285935541702, 'k': 3}
{'C': 1.2946969722600201, 'k': 3}
{'C': 1.7042648064952854, 'k': 3}
{'C': 2.276965939325671, 'k': 3}
{'C': 0.5295681908427132, 'k': 3}
{'C': 1.4578012788210766, 'k': 3}
{'C': 1.2440532860608564, 'k': 3}


In [193]:
type(uniform(loc=0.5,scale=1))
isinstance(uniform(loc=0.5,scale=1),scipy.stats._distn_infrastructure.rv_frozen)

True

In [192]:
import scipy

In [17]:
for i in product(*dic.values()):
    print(i)
    dac={keys:values for keys,values in zip(dic.keys(),i)}
    print(dac)

(1, 2)
{'a': 1, 'b': 2}
(1, 3)
{'a': 1, 'b': 3}
(3, 2)
{'a': 3, 'b': 2}
(3, 3)
{'a': 3, 'b': 3}
(4, 2)
{'a': 4, 'b': 2}
(4, 3)
{'a': 4, 'b': 3}
(5, 2)
{'a': 5, 'b': 2}
(5, 3)
{'a': 5, 'b': 3}


In [40]:
A=np.diag(np.arange(10))
idx=np.array([1,3,5])
A[idx]

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 3, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 5, 0, 0, 0, 0]])

In [182]:
def présentation_résultat2(search,n):
    mask=search['rank_test_score']<=n
    params=list(compress(search['params'], list(mask)))
    mean_test_score=search['mean_test_score'][mask]
    a={}
    for i in range(mean_test_score.size):
        k=''
        for key, value in params[i].items():
            k+=" "+key+" "+str(value)
        a.update({k:mean_test_score[i]})
        sortedDict = sorted(a.items(), key=lambda x: x[1],reverse=True)
    l=[]
    for i in sortedDict:
        u=i[0].split(sep=' ')
        del(u[0])
        lp=[]
        for j in u[1::2]:
            lp.append(j)
        lp.append(i[1])
        l.append(lp)
    head=list(params[0].keys())+["mean_test_score"]

    return(pd.DataFrame(l,columns=head))
        

# Simple MKL

# Mismatch Kernel

In [214]:
X_0train=pd.read_csv('/Users/elliotmuller/Documents/M2/Kernel Methods/Data Challenge/Xtr0.csv', sep=';')['seq']
X_1train=pd.read_csv('/Users/elliotmuller/Documents/M2/Kernel Methods/Data Challenge/Xtr1.csv', sep=';')['seq']
X_2train=pd.read_csv('/Users/elliotmuller/Documents/M2/Kernel Methods/Data Challenge/Xtr2.csv', sep=';')['seq']

X_0test=pd.read_csv('/Users/elliotmuller/Documents/M2/Kernel Methods/Data Challenge/Xte0.csv', sep=';')['seq']
X_1test=pd.read_csv('/Users/elliotmuller/Documents/M2/Kernel Methods/Data Challenge/Xte1.csv', sep=';')['seq']
X_2test=pd.read_csv('/Users/elliotmuller/Documents/M2/Kernel Methods/Data Challenge/Xte2.csv', sep=';')['seq']

Y_0=pd.read_csv('/Users/elliotmuller/Documents/M2/Kernel Methods/Data Challenge/Ytr0.csv',sep=',')
Y_1=pd.read_csv('/Users/elliotmuller/Documents/M2/Kernel Methods/Data Challenge/Ytr1.csv',sep=',')
Y_2=pd.read_csv('/Users/elliotmuller/Documents/M2/Kernel Methods/Data Challenge/Ytr2.csv',sep=',')
Y_0train=np.where(Y_0["Bound"]==0,-1,Y_0["Bound"])
Y_1train=np.where(Y_1["Bound"]==0,-1,Y_1["Bound"])
Y_2train=np.where(Y_2["Bound"]==0,-1,Y_2["Bound"])

X_0train[0]

'TCCTGTGCACATCTGCACCCCTGTTGTGGCCACAAAATGATCCGGCACCACCCAGTGGGAGACGACAGAGGTGGCAATGGGGTGTCGGCTCTGACGCCTCC'

In [199]:
import itertools
from Bio import pairwise2

def build_dic_voc(k):
    return {''.join(v):i for i,v in enumerate(product("ACGT",repeat=k))}

def mismatch_kernel(X,k,m,voc):
    if m>k:
        return("Error, k needs to be greater than m")
    L=[]
    iterable=['A','T','C','G']
    for subset in list(itertools.product('ACGT', repeat=k)):   #Toutes les sous sequences de longueur k possibles
        subseq=''.join(subset)
        L+=[subseq]   #On les stocke dans L 
    K=np.zeros((len(X),len(L)))   #On cree la matrice K 
    for i in (range(len(X))):   #Une ligne pour chaque observation
        sequence=X_0train[i]
        kernel_obs=np.zeros(len(L))
        for (j,subseq) in zip(range(len(L)),L):   #On regarde le nombre d'occurences de chaque sous seq possible
            #x=sequence.count(subseq)
            #kernel_obs[j]=x
            for l in (range(len(sequence)-(k-1))):
                s1=sequence[l:l+k]
                s2=subseq
                if (sum(c1!=c2 for c1,c2 in zip(s1,s2))<=m):
                    kernel_obs[j]+=1
        kernel_obs=kernel_obs.reshape(1,-1)
        K[i,]=kernel_obs
    return(K)
        
K=mismatch_kernel(X_0train,3,0)


In [ ]:
def build_dic_voc(k):
    return {''.join(v):i for i,v in enumerate(product("ACGT",repeat=k))}

def mismatch_kernel(X,k,m,voc):
    if m>k:
        return("Error, k needs to be greater than m")
    phi_X=np.zeros(len(voc))   
    sequence=X
    for subseq in (voc.keys()):   #On regarde le nombre d'occurences de chaque sous seq possible
        for l in (range(len(sequence)-(k-1))):
            s1=sequence[l:l+k]
            s2=subseq
            if (sum(c1!=c2 for c1,c2 in zip(s1,s2))<=m):
                phi_X[j]+=1
    return(phi_X)
        
K=mismatch_kernel(X_0train,3,0)


In [242]:
#zone_interet=X[l:l+k]
k=5
x='ATCTG'
x=list(x)
voc=['A','C','G','T']
m=2
L=[]
for mm in (range(m,-1,-1)):
    if mm==0:
        L.append(x)
    for i in itertools.combinations(range(k),mm):
        for y in (list(i)):
            x_copy=x.copy()
            voc_copy=voc[:]
            for u in voc_copy.remove[x[y]]:
                x_copy=x.copy()
                
        
    
print(x)  


['Z', 'Z', 'Z', 'Z', 'Z']


In [249]:
voc=['A','C','G','T']
voc.remove('A',inplace=False)
print(voc)

TypeError: remove() takes no keyword arguments

In [227]:
a=list(itertools.combinations(range(5),2))
len(a)
#x='ATCTG'
#for i in 


10

In [213]:
for k in (range(2,4)):
    for m in (range(k)):
        K=mismatch_kernel(X_0train,k,m)
        A=K.dot(K.T)
        print(np.linalg.matrix_rank(A))

16
16
64
64
64


In [191]:
import time 

a=time.time()

for k in (2,6):
    for m in (range(k)):
        K=mismatch_kernel(X_0train,2,0)
        A=K.dot(K.T)
        u=np.linalg.eigvals(A)
        a=0
        print(len(u))
        for i in range(len(u)):
            a+=(u[i]>0)
            print(a)
        if a==len(u):
            print('True for k={} and m={}'.format(k,m))
    b=time.time()
    print(b-a)

2000
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
22
22
22
22
22
22
23
23
24
25
25
25
25
25
26
27
27
27
27
27
28
29
30
31
32
33
33
33
34
35
36
37
38
39
39
39
39
39
40
41
42
43
43
43
43
43
43
43
43
43
44
45
45
45
45
45
45
45
46
47
47
47
48
49
50
51
52
53
54
55
56
57
58
59
59
59
60
61
62
63
63
63
63
63
64
65
65
65
66
67
68
69
70
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
71
72
72
72
72
72
72
72
72
72
72
72
72
72
73
74
74
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
88
88
88
88
89
90
91
92
93
94
94
94
95
96
97
98
99
100
101
102
103
104
105
106
106
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
124
124
124
124
124
124
124
124
124
124
124
124
124
124
125
126
126
126
126
126
127
128
129
130
130
130
130
130
130
130
130
130
130
130
131
132
132
132
132
132
133
134
134
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
152
152
152
152
152
152
152
152
153
154
154
154
154
154
154
154
154
154
154
154
154


808
808
808
808
809
810
810
810
810
810
810
810
810
810
810
810
810
810
810
810
810
810
810
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
846
846
846
846
846
846
847
848
849
850
851
852
853
854
854
854
854
854
854
854
854
854
854
854
855
856
857
858
858
858
858
859
860
860
860
861
862
863
864
865
866
867
868
869
870
871
871
871
872
873
874
875
875
875
875
875
875
875
875
875
876
877
877
877
877
877
877
877
877
877
877
877
877
877
878
879
879
879
880
881
881
881
882
883
883
883
883
883
884
885
885
885
886
887
888
889
890
891
892
893
893
893
893
893
893
893
894
895
896
897
897
897
897
897
898
899
899
899
899
899
900
901
902
903
903
903
903
903
903
903
904
905
906
907
907
907
907
907
907
907
907
907
908
909
910
911
911
911
912
913
914
915
916
917
918
919
919
919
919
919
920
921
922
923
924
925
926
927
928
929
930
930
930
930
930
930
930
931
932
932
932
932
932
932
932
933
934
934
934
934


KeyboardInterrupt: 

In [208]:

A=K.dot(K.T)
A.shape
#u=np.linalg.eigvals(A)
#a=0
#len(u)
#for i in range(64):
#    a+=(u[i]>0)
#a

(2000, 2000)

In [207]:
np.linalg.matrix_rank(A)

64

In [150]:
import itertools
iterable=['ACGT']
L=[]
for subset in list(itertools.product('ACGT', repeat=3)):
    subseq=''.join(subset)
    L+=[subseq]
print(L,len(L))

['AAA', 'AAC', 'AAG', 'AAT', 'ACA', 'ACC', 'ACG', 'ACT', 'AGA', 'AGC', 'AGG', 'AGT', 'ATA', 'ATC', 'ATG', 'ATT', 'CAA', 'CAC', 'CAG', 'CAT', 'CCA', 'CCC', 'CCG', 'CCT', 'CGA', 'CGC', 'CGG', 'CGT', 'CTA', 'CTC', 'CTG', 'CTT', 'GAA', 'GAC', 'GAG', 'GAT', 'GCA', 'GCC', 'GCG', 'GCT', 'GGA', 'GGC', 'GGG', 'GGT', 'GTA', 'GTC', 'GTG', 'GTT', 'TAA', 'TAC', 'TAG', 'TAT', 'TCA', 'TCC', 'TCG', 'TCT', 'TGA', 'TGC', 'TGG', 'TGT', 'TTA', 'TTC', 'TTG', 'TTT'] 64
